In [ ]:
import pandas as pd

In [ ]:
transaction = pd.read_csv('Transactions2.csv', sep=';', dtype={'recipient':str, 'committer':str})
transaction.rename(columns={'Unnamed: 0': 'ID', 'amunt':'amount'}, inplace=True)
transaction.set_index('ID')
subject = pd.read_excel('subjects.xlsx',dtype={'ID':str,'OLDID':str})
# subject.shape
# transaction.head()


,ID,amount,recipient,committer,type
0,0,3450,872,516,bank transfer
1,1,505,989,NaN,credit card
2,2,11320,178,702,bank transfer
3,3,9325,729,378,cheque
4,4,620,706,NaN,credit card


In [ ]:
transaction1 = pd.merge(transaction,subject,left_on='committer',right_on='ID')
transaction2 = pd.merge(transaction,subject,left_on='recipient',right_on='ID')


(206, 18)

## Adding the 'largecommitter' variable to the subjects dataset

In [ ]:
condition = transaction1.groupby('committer')['committer'].count() > 3
cond_frm = condition.to_frame(name="largecomitter")
pd.merge(transaction1, cond_frm, on="committer")


,ID_x,amount,recipient,committer,type,OLDID,ID_y,LASTNAME,FIRSTNAME,BIRTHDAY,BIRTHMONTH,BIRTHYEAR,GENDER,STREET ADDRESS,CAP,CITY,LATITUDE,LONGITUDE,largecomitter
0,0,3450,872,516,bank transfer,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,True
1,183,6945,689,516,bank transfer,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,True
2,369,16755,860,516,cheque,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,True
3,1912,8495,557,516,bank transfer,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,True
4,4442,5375,162,516,bank transfer,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,5000,32035,624,842,bank transfer,841,842,Matteuzzi,Cesarina,27,4,1984,Female,"Via Colonnello Elia, 34",73020,MELENDUGNO,40.241873,18.377608,True
202,5273,12950,804,842,cheque,841,842,Matteuzzi,Cesarina,27,4,1984,Female,"Via Colonnello Elia, 34",73020,MELENDUGNO,40.241873,18.377608,True
203,5443,58680,840,842,bank transfer,841,842,Matteuzzi,Cesarina,27,4,1984,Female,"Via Colonnello Elia, 34",73020,MELENDUGNO,40.241873,18.377608,True
204,5463,2030,111,842,bank transfer,841,842,Matteuzzi,Cesarina,27,4,1984,Female,"Via Colonnello Elia, 34",73020,MELENDUGNO,40.241873,18.377608,True


## Introducing the charge variable to transaction2 dataset

In [ ]:
transaction2.loc[transaction2['type'] == 'credit card', 'charge'] = transaction2['amount']*0.03
transaction2.loc[transaction2['type'] == 'bank transfer', 'charge'] = 3
transaction2.loc[(transaction2['type'] == 'cheque') | (transaction2['type'] == 'cash'), 'charge'] = 0
transaction2.tail()

,ID_x,amount,recipient,committer,type,OLDID,ID_y,LASTNAME,FIRSTNAME,BIRTHDAY,BIRTHMONTH,BIRTHYEAR,GENDER,STREET ADDRESS,CAP,CITY,LATITUDE,LONGITUDE,charge
274,4559,820,194,NaN,credit card,193,194,Bentivogli,Gabriele,28,6,1987,Male,"Via G. Cesare, 70",80125,NAPOLI,40.850696,14.246317,24.6
275,4683,6040,194,521,bank transfer,193,194,Bentivogli,Gabriele,28,6,1987,Male,"Via G. Cesare, 70",80125,NAPOLI,40.850696,14.246317,3.0
276,4973,305,194,866,cash,193,194,Bentivogli,Gabriele,28,6,1987,Male,"Via G. Cesare, 70",80125,NAPOLI,40.850696,14.246317,0.0
277,5130,7239,194,889,bank transfer,193,194,Bentivogli,Gabriele,28,6,1987,Male,"Via G. Cesare, 70",80125,NAPOLI,40.850696,14.246317,3.0
278,5194,4095,516,585,bank transfer,515,516,Rossetti,Domenico,28,2,1985,Male,"Via Trieste, 20",20092,CINISELLO BALSAMO,45.557393,9.218068,3.0


## Sum of transaction and taxes

In [ ]:
from numpy import reciprocal
report = transaction2.groupby(['recipient']).agg({'recipient':['count'],'charge':['sum']})


,recipient,charge
,count,sum
recipient,,
127,3,24.60
136,2,0.00
143,7,9.00
178,4,9.00
194,4,30.60
239,6,24.90
247,14,80.25
275,9,53.85


## Save the report in a csv file


In [ ]:
total_tax = transaction2.groupby(['recipient'])['charge'].sum()
tax = total_tax.to_frame(name='total_tax')
t2 = transaction2.drop_duplicates(subset=['recipient'])
report2 = pd.merge(t2, tax, on='recipient')
report2[['ID_y', 'FIRSTNAME', 'LASTNAME', 'total_tax']]

report2.to_excel('report.xlsx')